In [78]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 200

import numpy as np

from pandas.plotting import scatter_matrix
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
df = pd.read_csv('data/Seasons_stats_complete.csv')

In [80]:
drop_me = ['Unnamed: 0', 'Year', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'OWS', 'DWS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'FG%', '3P%', '2P%',
       'eFG%', 'FT%', 'ORB', 'DRB']

In [81]:
df_2011 = df[df['Year']==2011].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2012 = df[df['Year']==2012].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2013 = df[df['Year']==2013].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2014 = df[df['Year']==2014].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2015 = df[df['Year']==2015].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2016 = df[df['Year']==2016].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2017 = df[df['Year']==2017].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')
df_2018 = df[df['Year']==2018].drop(columns=drop_me).drop_duplicates('Player').reset_index().drop(columns='index')

In [82]:
df_2011

,Player,Pos,Age,Tm,G,MP,USG%,WS,FG,FGA,3P,3PA,2P,2PA,FT,FTA,TRB,AST,STL,BLK,TOV,PF,PTS
0,Jeff Adrien,PF,24.0,GSW,23.0,196.0,16.1,0.2,23.0,54.0,0.0,0.0,23.0,54.0,11.0,19.0,58.0,10.0,4.0,5.0,9.0,28.0,57.0
1,Arron Afflalo,SG,25.0,DEN,69.0,2324.0,14.8,6.2,312.0,627.0,105.0,248.0,207.0,379.0,138.0,163.0,251.0,168.0,34.0,31.0,71.0,153.0,867.0
2,Maurice Ager,SG,26.0,MIN,4.0,29.0,22.0,0.0,6.0,11.0,3.0,4.0,3.0,7.0,0.0,0.0,2.0,1.0,1.0,0.0,4.0,4.0,15.0
3,Alexis Ajinca,C,22.0,TOT,34.0,340.0,21.1,0.4,59.0,133.0,12.0,34.0,47.0,99.0,13.0,18.0,78.0,10.0,11.0,19.0,18.0,73.0,143.0
4,Solomon Alabi,C,22.0,TOR,12.0,59.0,14.3,-0.2,3.0,15.0,0.0,0.0,3.0,15.0,0.0,2.0,14.0,2.0,2.0,2.0,3.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Dorell Wright,SF,25.0,GSW,82.0,3147.0,19.3,5.8,485.0,1146.0,194.0,516.0,291.0,630.0,180.0,228.0,431.0,242.0,124.0,63.0,133.0,171.0,1344.0
448,Julian Wright,SF,23.0,TOR,52.0,766.0,13.0,0.6,82.0,160.0,2.0,10.0,80.0,150.0,22.0,43.0,118.0,58.0,39.0,20.0,43.0,45.0,188.0
449,Nick Young,SG,25.0,WAS,64.0,2034.0,24.4,2.8,412.0,935.0,105.0,271.0,307.0,664.0,186.0,228.0,174.0,75.0,45.0,17.0,90.0,144.0,1115.0
450,Sam Young,SF,25.0,MEM,78.0,1577.0,17.3,2.9,233.0,494.0,17.0,50.0,216.0,444.0,89.0,116.0,187.0,71.0,68.0,23.0,61.0,120.0,572.0


In [83]:
merged = df_2011.set_index('Player')\
    .join(df_2012.set_index('Player'),lsuffix='_2011', rsuffix='_2012',how='outer')\
    .join(df_2013.set_index('Player'),lsuffix='_2012', rsuffix='_2013',how='outer')\
    .join(df_2014.set_index('Player'),lsuffix='_2013', rsuffix='_2014',how='outer')\
    .join(df_2015.set_index('Player'),lsuffix='_2014', rsuffix='_2015',how='outer')\
    .join(df_2016.set_index('Player'),lsuffix='_2015', rsuffix='_2016',how='outer')\
    .join(df_2017.set_index('Player'),lsuffix='_2016', rsuffix='_2017',how='outer')\
    .join(df_2018.set_index('Player'),lsuffix='_2017', rsuffix='_2018',how='outer').reset_index()

In [84]:
merged

,Player,Pos_2011,Age_2011,Tm_2011,G_2011,MP_2011,USG%_2011,WS_2011,FG_2011,FGA_2011,3P_2011,3PA_2011,2P_2011,2PA_2011,FT_2011,FTA_2011,TRB_2011,AST_2011,STL_2011,BLK_2011,TOV_2011,PF_2011,PTS_2011,Pos_2012,Age_2012,Tm_2012,G_2012,MP_2012,USG%_2012,WS_2012,FG_2012,FGA_2012,3P_2012,3PA_2012,2P_2012,2PA_2012,FT_2012,FTA_2012,TRB_2012,AST_2012,STL_2012,BLK_2012,TOV_2012,PF_2012,PTS_2012,Pos_2013,Age_2013,Tm_2013,G_2013,MP_2013,USG%_2013,WS_2013,FG_2013,FGA_2013,3P_2013,3PA_2013,2P_2013,2PA_2013,FT_2013,FTA_2013,TRB_2013,AST_2013,STL_2013,BLK_2013,TOV_2013,PF_2013,PTS_2013,Pos_2014,Age_2014,Tm_2014,G_2014,MP_2014,USG%_2014,WS_2014,FG_2014,FGA_2014,3P_2014,3PA_2014,2P_2014,2PA_2014,FT_2014,FTA_2014,TRB_2014,AST_2014,STL_2014,BLK_2014,TOV_2014,PF_2014,PTS_2014,Pos_2015,Age_2015,Tm_2015,G_2015,MP_2015,USG%_2015,WS_2015,FG_2015,FGA_2015,3P_2015,3PA_2015,2P_2015,2PA_2015,FT_2015,FTA_2015,TRB_2015,AST_2015,STL_2015,BLK_2015,TOV_2015,PF_2015,PTS_2015,Pos_2016,Age_2016,Tm_2016,G_2016,MP_2016,USG%_2016,WS_2016,FG_2016,FGA_2016,3P_2016,3PA_2016,2P_2016,2PA_2016,FT_2016,FTA_2016,TRB_2016,AST_2016,STL_2016,BLK_2016,TOV_2016,PF_2016,PTS_2016,Pos_2017,Age_2017,Tm_2017,G_2017,MP_2017,USG%_2017,WS_2017,FG_2017,FGA_2017,3P_2017,3PA_2017,2P_2017,2PA_2017,FT_2017,FTA_2017,TRB_2017,AST_2017,STL_2017,BLK_2017,TOV_2017,PF_2017,PTS_2017,Pos_2018,Age_2018,Tm_2018,G_2018,MP_2018,USG%_2018,WS_2018,FG_2018,FGA_2018,3P_2018,3PA_2018,2P_2018,2PA_2018,FT_2018,FTA_2018,TRB_2018,AST_2018,STL_2018,BLK_2018,TOV_2018,PF_2018,PTS_2018
0,A.J. Hammons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,24.0,DAL,22.0,163.0,17.6,0.0,17.0,42.0,5.0,10.0,12.0,32.0,9.0,20.0,36.0,4.0,1.0,13.0,10.0,21.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Price,PG,24.0,IND,50.0,795.0,22.7,0.3,114.0,320.0,41.0,149.0,73.0,171.0,54.0,81.0,72.0,111.0,29.0,1.0,53.0,61.0,323.0,PG,25.0,IND,44.0,568.0,17.7,0.7,59.0,174.0,26.0,88.0,33.0,86.0,28.0,35.0,61.0,86.0,20.0,2.0,32.0,30.0,172.0,PG,26.0,WAS,57.0,1278.0,18.0,2.2,161.0,413.0,70.0,200.0,91.0,213.0,49.0,62.0,114.0,205.0,33.0,3.0,64.0,73.0,441.0,SG,27.0,MIN,28.0,99.0,23.2,0.0,19.0,46.0,6.0,22.0,13.0,24.0,0.0,2.0,10.0,13.0,1.0,0.0,7.0,5.0,44.0,PG,28.0,TOT,26.0,324.0,22.5,0.3,51.0,137.0,15.0,57.0,36.0,80.0,16.0,24.0,32.0,46.0,7.0,0.0,14.0,15.0,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Brooks,PG,26.0,TOT,59.0,1284.0,25.9,1.1,220.0,587.0,70.0,236.0,150.0,351.0,124.0,140.0,78.0,233.0,34.0,3.0,99.0,115.0,634.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PG,28.0,TOT,53.0,997.0,18.1,0.8,143.0,316.0,50.0,134.0,93.0,182.0,40.0,52.0,80.0,114.0,30.0,10.0,67.0,94.0,376.0,PG,29.0,TOT,72.0,1557.0,20.5,2.2,233.0,581.0,96.0,248.0,137.0,333.0,83.0,95.0,140.0,233.0,52.0,13.0,117.0,146.0,645.0,PG,30.0,CHI,82.0,1885.0,25.0,3.3,344.0,817.0,121.0,313.0,223.0,504.0,145.0,174.0,166.0,261.0,54.0,15.0,157.0,189.0,954.0,PG,31.0,CHI,69.0,1108.0,22.9,0.9,188.0,469.0,66.0,185.0,122.0,284.0,49.0,64.0,101.0,180.0,30.0,10.0,82.0,132.0,491.0,PG,32.0,IND,65.0,894.0,19.2,0.3,121.0,300.0,48.0,128.0,73.0,172.0,32.0,40.0,69.0,125.0,25.0,9.0,66.0,93.0,322.0,PG,33.0,MIN,32.0,189.0,19.9,0.1,28.0,69.0,11.0,31.0,17.0,38.0,8.0,11.0,17.0,20.0,6.0,0.0,11.0,28.0,75.0
3,Aaron Gordon,NaN,NaN,Na

In [85]:
merged_train = merged.dropna(subset=['TRB_2017', 'TRB_2018'])

In [86]:
merged_train

,Player,Pos_2011,Age_2011,Tm_2011,G_2011,MP_2011,USG%_2011,WS_2011,FG_2011,FGA_2011,3P_2011,3PA_2011,2P_2011,2PA_2011,FT_2011,FTA_2011,TRB_2011,AST_2011,STL_2011,BLK_2011,TOV_2011,PF_2011,PTS_2011,Pos_2012,Age_2012,Tm_2012,G_2012,MP_2012,USG%_2012,WS_2012,FG_2012,FGA_2012,3P_2012,3PA_2012,2P_2012,2PA_2012,FT_2012,FTA_2012,TRB_2012,AST_2012,STL_2012,BLK_2012,TOV_2012,PF_2012,PTS_2012,Pos_2013,Age_2013,Tm_2013,G_2013,MP_2013,USG%_2013,WS_2013,FG_2013,FGA_2013,3P_2013,3PA_2013,2P_2013,2PA_2013,FT_2013,FTA_2013,TRB_2013,AST_2013,STL_2013,BLK_2013,TOV_2013,PF_2013,PTS_2013,Pos_2014,Age_2014,Tm_2014,G_2014,MP_2014,USG%_2014,WS_2014,FG_2014,FGA_2014,3P_2014,3PA_2014,2P_2014,2PA_2014,FT_2014,FTA_2014,TRB_2014,AST_2014,STL_2014,BLK_2014,TOV_2014,PF_2014,PTS_2014,Pos_2015,Age_2015,Tm_2015,G_2015,MP_2015,USG%_2015,WS_2015,FG_2015,FGA_2015,3P_2015,3PA_2015,2P_2015,2PA_2015,FT_2015,FTA_2015,TRB_2015,AST_2015,STL_2015,BLK_2015,TOV_2015,PF_2015,PTS_2015,Pos_2016,Age_2016,Tm_2016,G_2016,MP_2016,USG%_2016,WS_2016,FG_2016,FGA_2016,3P_2016,3PA_2016,2P_2016,2PA_2016,FT_2016,FTA_2016,TRB_2016,AST_2016,STL_2016,BLK_2016,TOV_2016,PF_2016,PTS_2016,Pos_2017,Age_2017,Tm_2017,G_2017,MP_2017,USG%_2017,WS_2017,FG_2017,FGA_2017,3P_2017,3PA_2017,2P_2017,2PA_2017,FT_2017,FTA_2017,TRB_2017,AST_2017,STL_2017,BLK_2017,TOV_2017,PF_2017,PTS_2017,Pos_2018,Age_2018,Tm_2018,G_2018,MP_2018,USG%_2018,WS_2018,FG_2018,FGA_2018,3P_2018,3PA_2018,2P_2018,2PA_2018,FT_2018,FTA_2018,TRB_2018,AST_2018,STL_2018,BLK_2018,TOV_2018,PF_2018,PTS_2018
2,Aaron Brooks,PG,26.0,TOT,59.0,1284.0,25.9,1.1,220.0,587.0,70.0,236.0,150.0,351.0,124.0,140.0,78.0,233.0,34.0,3.0,99.0,115.0,634.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PG,28.0,TOT,53.0,997.0,18.1,0.8,143.0,316.0,50.0,134.0,93.0,182.0,40.0,52.0,80.0,114.0,30.0,10.0,67.0,94.0,376.0,PG,29.0,TOT,72.0,1557.0,20.5,2.2,233.0,581.0,96.0,248.0,137.0,333.0,83.0,95.0,140.0,233.0,52.0,13.0,117.0,146.0,645.0,PG,30.0,CHI,82.0,1885.0,25.0,3.3,344.0,817.0,121.0,313.0,223.0,504.0,145.0,174.0,166.0,261.0,54.0,15.0,157.0,189.0,954.0,PG,31.0,CHI,69.0,1108.0,22.9,0.9,188.0,469.0,66.0,185.0,122.0,284.0,49.0,64.0,101.0,180.0,30.0,10.0,82.0,132.0,491.0,PG,32.0,IND,65.0,894.0,19.2,0.3,121.0,300.0,48.0,128.0,73.0,172.0,32.0,40.0,69.0,125.0,25.0,9.0,66.0,93.0,322.0,PG,33.0,MIN,32.0,189.0,19.9,0.1,28.0,69.0,11.0,31.0,17.0,38.0,8.0,11.0,17.0,20.0,6.0,0.0,11.0,28.0,75.0
3,Aaron Gordon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF,19.0,ORL,47.0,797.0,15.5,1.0,93.0,208.0,13.0,48.0,80.0,160.0,44.0,61.0,169.0,33.0,21.0,22.0,38.0,83.0,243.0,PF,20.0,ORL,78.0,1863.0,17.3,5.4,274.0,579.0,42.0,142.0,232.0,437.0,129.0,193.0,507.0,128.0,59.0,55.0,66.0,153.0,719.0,SF,21.0,ORL,80.0,2298.0,20.1,3.7,393.0,865.0,77.0,267.0,316.0,598.0,156.0,217.0,405.0,150.0,64.0,40.0,89.0,172.0,1019.0,PF,22.0,ORL,58.0,1909.0,24.7,2.9,375.0,865.0,115.0,342.0,260.0,523.0,157.0,225.0,457.0,136.0,59.0,45.0,107.0,111.0,1022.0
5,Aaron Harrison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SG,21.0,CHO,21.0,93.0,13.7,0.0,5.0,19.0,3.0,10.0,2.0,9.0,5.0,12.0,15.0,2.0,6.0,0.0,4.0,10.0,18.0,SG,22.0,CHO,5.0,17.0,12.9,-0.1,0.0,4.0,0.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,2.0,1.0,SG,23.0,DAL,9.0,233.0,15.5,-0.1,19.0,69.0,9.0,43.0,10.0,26.0,13.0,17.0,24.0,11.0,9.0,2.0,3.

In [87]:
merged_train = merged_train.fillna(0)

In [88]:
columns = merged_train.columns

In [89]:
columns

Index(['Player', 'Pos_2011', 'Age_2011', 'Tm_2011', 'G_2011', 'MP_2011',
       'USG%_2011', 'WS_2011', 'FG_2011', 'FGA_2011', '3P_2011', '3PA_2011',
       '2P_2011', '2PA_2011', 'FT_2011', 'FTA_2011', 'TRB_2011', 'AST_2011',
       'STL_2011', 'BLK_2011', 'TOV_2011', 'PF_2011', 'PTS_2011', 'Pos_2012',
       'Age_2012', 'Tm_2012', 'G_2012', 'MP_2012', 'USG%_2012', 'WS_2012',
       'FG_2012', 'FGA_2012', '3P_2012', '3PA_2012', '2P_2012', '2PA_2012',
       'FT_2012', 'FTA_2012', 'TRB_2012', 'AST_2012', 'STL_2012', 'BLK_2012',
       'TOV_2012', 'PF_2012', 'PTS_2012', 'Pos_2013', 'Age_2013', 'Tm_2013',
       'G_2013', 'MP_2013', 'USG%_2013', 'WS_2013', 'FG_2013', 'FGA_2013',
       '3P_2013', '3PA_2013', '2P_2013', '2PA_2013', 'FT_2013', 'FTA_2013',
       'TRB_2013', 'AST_2013', 'STL_2013', 'BLK_2013', 'TOV_2013', 'PF_2013',
       'PTS_2013', 'Pos_2014', 'Age_2014', 'Tm_2014', 'G_2014', 'MP_2014',
       'USG%_2014', 'WS_2014', 'FG_2014', 'FGA_2014', '3P_2014', '3PA_2014',
       '

In [90]:
drop_me = ['Player', 'Pos_2011', 'Tm_2011', 'WS_2011', 'Pos_2012',
       'Tm_2012', 'WS_2012',
       'Pos_2013', 'Tm_2013', 'WS_2013', 'Pos_2014', 'Tm_2014', 'WS_2014', 'Pos_2015',
       'Tm_2015', 'WS_2015', 'Pos_2016', 'Tm_2016', 'WS_2016', 'Pos_2017', 'Tm_2017', 'WS_2017','Pos_2018',
       'Age_2018', 'Tm_2018', 'G_2018', 'MP_2018', 'USG%_2018', 'WS_2018',
       'FG_2018', 'FGA_2018', '3P_2018', '3PA_2018', '2P_2018', '2PA_2018',
       'FT_2018', 'FTA_2018', 'TRB_2018', 'AST_2018', 'STL_2018', 'BLK_2018',
       'TOV_2018', 'PF_2018', 'PTS_2018']
merged_train_X = merged_train.drop(columns=drop_me)
merged_train_y = merged_train['WS_2018']

In [91]:
merged_train_X

,Age_2011,G_2011,MP_2011,USG%_2011,FG_2011,FGA_2011,3P_2011,3PA_2011,2P_2011,2PA_2011,FT_2011,FTA_2011,TRB_2011,AST_2011,STL_2011,BLK_2011,TOV_2011,PF_2011,PTS_2011,Age_2012,G_2012,MP_2012,USG%_2012,FG_2012,FGA_2012,3P_2012,3PA_2012,2P_2012,2PA_2012,FT_2012,FTA_2012,TRB_2012,AST_2012,STL_2012,BLK_2012,TOV_2012,PF_2012,PTS_2012,Age_2013,G_2013,MP_2013,USG%_2013,FG_2013,FGA_2013,3P_2013,3PA_2013,2P_2013,2PA_2013,FT_2013,FTA_2013,TRB_2013,AST_2013,STL_2013,BLK_2013,TOV_2013,PF_2013,PTS_2013,Age_2014,G_2014,MP_2014,USG%_2014,FG_2014,FGA_2014,3P_2014,3PA_2014,2P_2014,2PA_2014,FT_2014,FTA_2014,TRB_2014,AST_2014,STL_2014,BLK_2014,TOV_2014,PF_2014,PTS_2014,Age_2015,G_2015,MP_2015,USG%_2015,FG_2015,FGA_2015,3P_2015,3PA_2015,2P_2015,2PA_2015,FT_2015,FTA_2015,TRB_2015,AST_2015,STL_2015,BLK_2015,TOV_2015,PF_2015,PTS_2015,Age_2016,G_2016,MP_2016,USG%_2016,FG_2016,FGA_2016,3P_2016,3PA_2016,2P_2016,2PA_2016,FT_2016,FTA_2016,TRB_2016,AST_2016,STL_2016,BLK_2016,TOV_2016,PF_2016,PTS_2016,Age_2017,G_2017,MP_2017,USG%_2017,FG_2017,FGA_2017,3P_2017,3PA_2017,2P_2017,2PA_2017,FT_2017,FTA_2017,TRB_2017,AST_2017,STL_2017,BLK_2017,TOV_2017,PF_2017,PTS_2017
2,26.0,59.0,1284.0,25.9,220.0,587.0,70.0,236.0,150.0,351.0,124.0,140.0,78.0,233.0,34.0,3.0,99.0,115.0,634.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,53.0,997.0,18.1,143.0,316.0,50.0,134.0,93.0,182.0,40.0,52.0,80.0,114.0,30.0,10.0,67.0,94.0,376.0,29.0,72.0,1557.0,20.5,233.0,581.0,96.0,248.0,137.0,333.0,83.0,95.0,140.0,233.0,52.0,13.0,117.0,146.0,645.0,30.0,82.0,1885.0,25.0,344.0,817.0,121.0,313.0,223.0,504.0,145.0,174.0,166.0,261.0,54.0,15.0,157.0,189.0,954.0,31.0,69.0,1108.0,22.9,188.0,469.0,66.0,185.0,122.0,284.0,49.0,64.0,101.0,180.0,30.0,10.0,82.0,132.0,491.0,32.0,65.0,894.0,19.2,121.0,300.0,48.0,128.0,73.0,172.0,32.0,40.0,69.0,125.0,25.0,9.0,66.0,93.0,322.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,47.0,797.0,15.5,93.0,208.0,13.0,48.0,80.0,160.0,44.0,61.0,169.0,33.0,21.0,22.0,38.0,83.0,243.0,20.0,78.0,1863.0,17.3,274.0,579.0,42.0,142.0,232.0,437.0,129.0,193.0,507.0,128.0,59.0,55.0,66.0,153.0,719.0,21.0,80.0,2298.0,20.1,393.0,865.0,77.0,267.0,316.0,598.0,156.0,217.0,405.0,150.0,64.0,40.0,89.0,172.0,1019.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,21.0,93.0,13.7,5.0,19.0,3.0,10.0,2.0,9.0,5.0,12.0,15.0,2.0,6.0,0.0,4.0,10.0,18.0,22.0,5.0,17.0,12.9,0.0,4.0,0.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,2.0,1.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,32.0,739.0,17.0,91.0,220.0,1.0,9.0,90.0,211.0,30.0,46.0,162.0,30.0,19.0,9.0,44.0,88.0,213.0,24.0,52.0,486.0,17.7,53.0,145.0,9.0,32.0,44.0,113.0,17.0,26.0,111.0,29.0,16.0,11.0,36.0,77.0,132.0,25.0,18.0,135.0,23.1,23.0,54.0,3.0,15.0,20.0,39.0,14.0,19.0,33.0,7.0,8.0,7.0,8.0,32.0,63.0
12,24.0,77.0,2704.0,19.7,513.0,921.0,2.0,4.0,511.0,917.0,150.0,188.0,718.0,266.0,59.0,80.0,119.0,193.0,1178.0,25.0,11.0,348.0,17.9,57.0,103.0,0.0,1.0,57.0,102.0,22.0,30.0,77.0,24.0,10.0,14.0,16.0,21.0,136.0,26.0,74.0,2756.0,21.8,576.0,1060.0,3.0,6.0,573.0,1054.0,134.0,208.0,757.0,240.0,78.0,78.0,147.0,163.0,1289.0,27.0,29.0,958.0,24.7,238.0,420.0,4.0,11.0,234.0,409.0,58.0,85

In [169]:
merged_for_predict = merged.drop(merged.filter(regex='2011').columns, axis=1)

In [170]:
merged_for_predict = merged_for_predict.dropna(subset=['TRB_2018'])

In [171]:
merged_for_predict = merged_for_predict.fillna(0)

In [172]:
drop_me = ['Player', 'Pos_2012',
       'Tm_2012', 'WS_2012',
       'Pos_2013', 'Tm_2013', 'WS_2013', 'Pos_2014', 'Tm_2014', 'WS_2014', 'Pos_2015',
       'Tm_2015', 'WS_2015', 'Pos_2016', 'Tm_2016', 'WS_2016', 'Pos_2017', 'Tm_2017', 'WS_2017','Pos_2018',
       'Tm_2018']

In [173]:
merged_for_predict = merged_for_predict.drop(columns=drop_me)
merged_for_predict

,Age_2012,G_2012,MP_2012,USG%_2012,FG_2012,FGA_2012,3P_2012,3PA_2012,2P_2012,2PA_2012,FT_2012,FTA_2012,TRB_2012,AST_2012,STL_2012,BLK_2012,TOV_2012,PF_2012,PTS_2012,Age_2013,G_2013,MP_2013,USG%_2013,FG_2013,FGA_2013,3P_2013,3PA_2013,2P_2013,2PA_2013,FT_2013,FTA_2013,TRB_2013,AST_2013,STL_2013,BLK_2013,TOV_2013,PF_2013,PTS_2013,Age_2014,G_2014,MP_2014,USG%_2014,FG_2014,FGA_2014,3P_2014,3PA_2014,2P_2014,2PA_2014,FT_2014,FTA_2014,TRB_2014,AST_2014,STL_2014,BLK_2014,TOV_2014,PF_2014,PTS_2014,Age_2015,G_2015,MP_2015,USG%_2015,FG_2015,FGA_2015,3P_2015,3PA_2015,2P_2015,2PA_2015,FT_2015,FTA_2015,TRB_2015,AST_2015,STL_2015,BLK_2015,TOV_2015,PF_2015,PTS_2015,Age_2016,G_2016,MP_2016,USG%_2016,FG_2016,FGA_2016,3P_2016,3PA_2016,2P_2016,2PA_2016,FT_2016,FTA_2016,TRB_2016,AST_2016,STL_2016,BLK_2016,TOV_2016,PF_2016,PTS_2016,Age_2017,G_2017,MP_2017,USG%_2017,FG_2017,FGA_2017,3P_2017,3PA_2017,2P_2017,2PA_2017,FT_2017,FTA_2017,TRB_2017,AST_2017,STL_2017,BLK_2017,TOV_2017,PF_2017,PTS_2017,Age_2018,G_2018,MP_2018,USG%_2018,WS_2018,FG_2018,FGA_2018,3P_2018,3PA_2018,2P_2018,2PA_2018,FT_2018,FTA_2018,TRB_2018,AST_2018,STL_2018,BLK_2018,TOV_2018,PF_2018,PTS_2018
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,53.0,997.0,18.1,143.0,316.0,50.0,134.0,93.0,182.0,40.0,52.0,80.0,114.0,30.0,10.0,67.0,94.0,376.0,29.0,72.0,1557.0,20.5,233.0,581.0,96.0,248.0,137.0,333.0,83.0,95.0,140.0,233.0,52.0,13.0,117.0,146.0,645.0,30.0,82.0,1885.0,25.0,344.0,817.0,121.0,313.0,223.0,504.0,145.0,174.0,166.0,261.0,54.0,15.0,157.0,189.0,954.0,31.0,69.0,1108.0,22.9,188.0,469.0,66.0,185.0,122.0,284.0,49.0,64.0,101.0,180.0,30.0,10.0,82.0,132.0,491.0,32.0,65.0,894.0,19.2,121.0,300.0,48.0,128.0,73.0,172.0,32.0,40.0,69.0,125.0,25.0,9.0,66.0,93.0,322.0,33.0,32.0,189.0,19.9,0.1,28.0,69.0,11.0,31.0,17.0,38.0,8.0,11.0,17.0,20.0,6.0,0.0,11.0,28.0,75.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,47.0,797.0,15.5,93.0,208.0,13.0,48.0,80.0,160.0,44.0,61.0,169.0,33.0,21.0,22.0,38.0,83.0,243.0,20.0,78.0,1863.0,17.3,274.0,579.0,42.0,142.0,232.0,437.0,129.0,193.0,507.0,128.0,59.0,55.0,66.0,153.0,719.0,21.0,80.0,2298.0,20.1,393.0,865.0,77.0,267.0,316.0,598.0,156.0,217.0,405.0,150.0,64.0,40.0,89.0,172.0,1019.0,22.0,58.0,1909.0,24.7,2.9,375.0,865.0,115.0,342.0,260.0,523.0,157.0,225.0,457.0,136.0,59.0,45.0,107.0,111.0,1022.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,21.0,93.0,13.7,5.0,19.0,3.0,10.0,2.0,9.0,5.0,12.0,15.0,2.0,6.0,0.0,4.0,10.0,18.0,22.0,5.0,17.0,12.9,0.0,4.0,0.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,2.0,1.0,23.0,9.0,233.0,15.5,-0.1,19.0,69.0,9.0,43.0,10.0,26.0,13.0,17.0,24.0,11.0,9.0,2.0,3.0,27.0,60.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,1.0,35.0,13.7,0.0,3.0,9.0,1.0,4.0,2.0,5.0,1.0,2.0,3.0,1.0,0.0,0.0,1.0,4.0,8.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [174]:
merged_for_predict_X = merged_for_predict.drop(columns='WS_2018')
merged_for_predict_y = merged_for_predict['WS_2018']

In [146]:
X_train, X_test, y_train, y_test = tts(merged_train_X,merged_train_y, test_size=.2)

In [165]:
rf = RandomForestRegressor(oob_score=True)
rf.fit(merged_train_X,merged_train_y)
rf.oob_score_

0.35761758906122154

In [177]:
rf_predict = rf.predict(merged_for_predict_X)

In [178]:
merged_for_predict['rf_predict'] = rf_predict
merged_for_predict['rf_predict-actual'] = rf_predict - merged_for_predict_y

In [179]:
merged_for_predict

,Age_2012,G_2012,MP_2012,USG%_2012,FG_2012,FGA_2012,3P_2012,3PA_2012,2P_2012,2PA_2012,FT_2012,FTA_2012,TRB_2012,AST_2012,STL_2012,BLK_2012,TOV_2012,PF_2012,PTS_2012,Age_2013,G_2013,MP_2013,USG%_2013,FG_2013,FGA_2013,3P_2013,3PA_2013,2P_2013,2PA_2013,FT_2013,FTA_2013,TRB_2013,AST_2013,STL_2013,BLK_2013,TOV_2013,PF_2013,PTS_2013,Age_2014,G_2014,MP_2014,USG%_2014,FG_2014,FGA_2014,3P_2014,3PA_2014,2P_2014,2PA_2014,FT_2014,FTA_2014,TRB_2014,AST_2014,STL_2014,BLK_2014,TOV_2014,PF_2014,PTS_2014,Age_2015,G_2015,MP_2015,USG%_2015,FG_2015,FGA_2015,3P_2015,3PA_2015,2P_2015,2PA_2015,FT_2015,FTA_2015,TRB_2015,AST_2015,STL_2015,BLK_2015,TOV_2015,PF_2015,PTS_2015,Age_2016,G_2016,MP_2016,USG%_2016,FG_2016,FGA_2016,3P_2016,3PA_2016,2P_2016,2PA_2016,FT_2016,FTA_2016,TRB_2016,AST_2016,STL_2016,BLK_2016,TOV_2016,PF_2016,PTS_2016,Age_2017,G_2017,MP_2017,USG%_2017,FG_2017,FGA_2017,3P_2017,3PA_2017,2P_2017,2PA_2017,FT_2017,FTA_2017,TRB_2017,AST_2017,STL_2017,BLK_2017,TOV_2017,PF_2017,PTS_2017,Age_2018,G_2018,MP_2018,USG%_2018,WS_2018,FG_2018,FGA_2018,3P_2018,3PA_2018,2P_2018,2PA_2018,FT_2018,FTA_2018,TRB_2018,AST_2018,STL_2018,BLK_2018,TOV_2018,PF_2018,PTS_2018,rf_predict,rf_predict-actual
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,53.0,997.0,18.1,143.0,316.0,50.0,134.0,93.0,182.0,40.0,52.0,80.0,114.0,30.0,10.0,67.0,94.0,376.0,29.0,72.0,1557.0,20.5,233.0,581.0,96.0,248.0,137.0,333.0,83.0,95.0,140.0,233.0,52.0,13.0,117.0,146.0,645.0,30.0,82.0,1885.0,25.0,344.0,817.0,121.0,313.0,223.0,504.0,145.0,174.0,166.0,261.0,54.0,15.0,157.0,189.0,954.0,31.0,69.0,1108.0,22.9,188.0,469.0,66.0,185.0,122.0,284.0,49.0,64.0,101.0,180.0,30.0,10.0,82.0,132.0,491.0,32.0,65.0,894.0,19.2,121.0,300.0,48.0,128.0,73.0,172.0,32.0,40.0,69.0,125.0,25.0,9.0,66.0,93.0,322.0,33.0,32.0,189.0,19.9,0.1,28.0,69.0,11.0,31.0,17.0,38.0,8.0,11.0,17.0,20.0,6.0,0.0,11.0,28.0,75.0,1.048,0.948
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,47.0,797.0,15.5,93.0,208.0,13.0,48.0,80.0,160.0,44.0,61.0,169.0,33.0,21.0,22.0,38.0,83.0,243.0,20.0,78.0,1863.0,17.3,274.0,579.0,42.0,142.0,232.0,437.0,129.0,193.0,507.0,128.0,59.0,55.0,66.0,153.0,719.0,21.0,80.0,2298.0,20.1,393.0,865.0,77.0,267.0,316.0,598.0,156.0,217.0,405.0,150.0,64.0,40.0,89.0,172.0,1019.0,22.0,58.0,1909.0,24.7,2.9,375.0,865.0,115.0,342.0,260.0,523.0,157.0,225.0,457.0,136.0,59.0,45.0,107.0,111.0,1022.0,4.816,1.916
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,21.0,93.0,13.7,5.0,19.0,3.0,10.0,2.0,9.0,5.0,12.0,15.0,2.0,6.0,0.0,4.0,10.0,18.0,22.0,5.0,17.0,12.9,0.0,4.0,0.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,2.0,1.0,23.0,9.0,233.0,15.5,-0.1,19.0,69.0,9.0,43.0,10.0,26.0,13.0,17.0,24.0,11.0,9.0,2.0,3.0,27.0,60.0,0.391,0.491
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,1.0,35.0,13.7,0.0,3.0,9.0,1.0,4.0,2.0,5.0,1.0,2.0,3.0,1.0,0.0,0.0,1.0,4.0,8.0,0.296,0.296
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [182]:
merged2 = merged.dropna(subset=['TRB_2018'])
merged2['rf_predict'] = rf_predict
merged2['rf_predict-actual'] = rf_predict - merged_for_predict_y

C:\Users\jooki\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jooki\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [185]:
hmm = merged2[['Player','Pos_2018','WS_2018','rf_predict','rf_predict-actual']]

In [188]:
hmm.sort_values('rf_predict',ascending=False).head(20)

,Player,Pos_2018,WS_2018,rf_predict,rf_predict-actual
658,LeBron James,PF,14.0,12.228,-1.772
893,Russell Westbrook,PG,10.1,11.055,0.955
372,Giannis Antetokounmpo,PF,11.9,11.052,-0.848
59,Anthony Davis,PF,13.7,10.455,-3.245
447,James Harden,SG,15.4,9.962,-5.438
206,Damian Lillard,PG,12.6,9.007,-3.593
592,Karl-Anthony Towns,C,14.0,8.987,-5.013
826,Paul George,SF,8.9,8.421,-0.479
251,DeMarcus Cousins,C,4.7,8.405,3.705
949,Stephen Curry,PG,9.1,8.225,-0.875
